# Brain teaser

[Original Github REPO](https://github.com/1171-jpg/BrainTeaser)


### Upute za pokretanje

1) Pokrenuti donju liniju i viditi jel se sve moze importati
1) Skinuti sve sto se ne moze sa `!pip install` unutar notebooka
(pip bi trebao raditi i s condom ako je dobro namjesteno)
    - Skinuti pytorch sa stranice [Pytorch](https://pytorch.org/get-started/locally/) - odaberite CUDA 11.8
    ako imate graficku. Meni je bilo:
    `> !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118`
    - Ako nemate CUDA, notebook ce biti sporiji - dovoljan za inference, 
    ali nedovoljan za fine-tuning (sreca pa to niti ne radimo, bilo bi presporo 
    i s grafickom)
    - Skinuti huggingface transformers library:
    `> pip install 'transformers[torch]'`
1) Moze se sve importati na Kaggle, ovo mozda malo kasnije

In [1]:
import numpy as np
import json
from datasets import load_dataset
from random import shuffle
import random

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import torch
from torch.nn.functional import normalize
from transformers import AutoModel, AutoTokenizer

from tqdm.auto import tqdm
import numpy as np
import os
from transformers import StoppingCriteriaList

In [117]:
letters = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P']
letters, len(letters)

(['A',
  'B',
  'C',
  'D',
  'E',
  'F',
  'G',
  'H',
  'I',
  'J',
  'K',
  'L',
  'M',
  'N',
  'O',
  'P'],
 16)

In [118]:
choice_to_index = {letters[i]: i for i in range(len(letters))}
choice_to_index[None] = len(letters)
choice_to_index

{'A': 0,
 'B': 1,
 'C': 2,
 'D': 3,
 'E': 4,
 'F': 5,
 'G': 6,
 'H': 7,
 'I': 8,
 'J': 9,
 'K': 10,
 'L': 11,
 'M': 12,
 'N': 13,
 'O': 14,
 'P': 15,
 None: 16}

In [119]:
import json
from tqdm import tqdm
import torch
import logging
import argparse
import numpy as np

def getResultdata(result_data):
    choice_to_index = {letters[i]: i for i in range(len(letters))}
    choice_to_index[None] = len(letters)

    word_play = {}
    reverse_play = {}
    for item in result_data:
        item_type = item['id'].split("-")[0]
        item_id = item['id'].split("-")[1].split("_")[0]
        if item_type == 'WP':
            if item_id not in word_play:
                word_play[item_id] = [0,0,0]
        else:
            if item_id not in reverse_play:
                reverse_play[item_id] = [0,0,0]

    for item in result_data:
        item_type = item['id'].split("-")[0]
        item_id = item['id'].split("-")[1].split("_")[0]
        ad_type = 0
        if 'SR' in item['id']:
            ad_type = 1
        elif 'CR' in item['id']:
            ad_type = 2
        else:
            ad_type = 0

        if item_type == 'WP':
            if choice_to_index[item['predict']] == item['label']:
                word_play[item_id][ad_type] = 1
        else:
            if choice_to_index[item['predict']] == item['label']:
                reverse_play[item_id][ad_type] = 1
                
    return word_play,reverse_play


def getMetric(data_list):
    data_list = np.array(data_list)
    overall_accuracy = np.sum(data_list)/3/len(data_list)
    original_accuracy = np.sum(data_list,axis = 0)[0]/len(data_list)
    semantic_accuracy = np.sum(data_list,axis = 0)[1]/len(data_list)
    context_accuracy = np.sum(data_list,axis = 0)[2]/len(data_list)
    ori_sema = np.sum([1 if item[0]==1 and item[1] == 1 else 0 for item in data_list])/len(data_list)
    ori_sema_cont = np.sum([1 if item[0]==1 and item[1] == 1 and item[2] == 1  else 0 for item in data_list])/len(data_list)
    
    print("over_all accuracy {}".format(overall_accuracy))
    print("single_original_accuracy {}".format(original_accuracy))
    print("single_semantic_accuracy {}".format(semantic_accuracy))
    print("single_context_accuracy {}".format(context_accuracy))
    print("sr_accuracy {}".format(ori_sema))
    print("cr_accuracy {}".format(ori_sema_cont))

    return {'over_all accuracy':overall_accuracy,'original_accuracy':original_accuracy,'semantic_accuracy':semantic_accuracy,'context_accuracy':context_accuracy,'ori_sema':ori_sema,'ori_sema_cont':ori_sema_cont}


def getSeperateResult(word_play,reverse_thinking):
    final_result = {}
    word_data_list = []
    word_data_list = list(word_play.values())
    print('#########Wordplay##########')
    final_result['wordplay'] = getMetric(word_data_list)
    
    reverse_data_list = []
    for item in reverse_thinking.values():
        reverse_data_list.append(item)
    print('#########Sentence##########')   
    final_result['sentence'] = getMetric(reverse_data_list)  
    
    
    all_data = word_data_list + reverse_data_list
    print('#########All data##########') 
    final_result['all'] = getMetric(all_data) 
    
    return final_result

In [120]:
def save(name=None, few_shot=False, example=format):
    try:
        os.mkdir('results')
    except:
        pass
    model_name = model_path.split('/')[-1]
    inserted_name = "" if name is None else "_" + name
    i = 1
    while True:
        file_path = f'results/{model_name}{inserted_name}_{i}.txt'
        try:
            with open(file_path, 'r'):
                i += 1
        except:
            with open(file_path, 'w') as file:
                file.write(model_name+'\n')
                if few_shot:
                    file.write(few_shot + '\n')
                if not isinstance(example, str):
                    example = json.dumps(example)
                file.write(example + '\n')
                file.write(json.dumps(final_result, indent=4))
                return

### Dataset

- Skinuti s [linka](https://drive.google.com/drive/u/0/folders/1kiFXp5fqpf8--NQJJAlBIBpfSaXTk1UY)

- Staviti u folder `/data`

In [121]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

### Load the model

In [122]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM
# model_path = 'google/flan-t5-xxl'
# model_path = 'google/flan-t5-xl'
# model_path = 'google/flan-t5-large'
# model_path = 'mistralai/Mistral-7B-Instruct-v0.2'
model_path = 'microsoft/phi-2'

tokenizer = AutoTokenizer.from_pretrained(model_path)

if model_path.find('flan') >= 0:
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
else:
    model = AutoModelForCausalLM.from_pretrained(model_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [123]:
try:
    model = model.to(device)
except Exception as e:
    print(e)
    print("Changing device to cpu")
    device = 'cpu'

In [124]:
# few_shot_examples = list(np.load("./{dataset_path}/data/demonstration.npy",allow_pickle=True))
data_path = '/kaggle/input/brainteaser/data'
sentence_data_path = f"{data_path}/SP-train.npy"
wordplay_data_list = f"{data_path}/WP-train.npy"
sentence_data_list = list(np.load(sentence_data_path,allow_pickle=True))
wordplay_data_list = list(np.load(wordplay_data_list,allow_pickle=True))

In [125]:
test_data_list = sentence_data_list + wordplay_data_list
print(f"Dataset length {len(test_data_list)}")

Dataset length 903


In [126]:
newline = '\n'
format = f"""Question: {"{}"}
Choice:
{''.join('(' + x + ') {}' + newline for x in letters)}
Answer:("""
print(format)

Question: {}
Choice:
(A) {}
(B) {}
(C) {}
(D) {}
(E) {}
(F) {}
(G) {}
(H) {}
(I) {}
(J) {}
(K) {}
(L) {}
(M) {}
(N) {}
(O) {}
(P) {}

Answer:(


In [127]:
def get_single_demo(sample):
    sample_demo = format.format(
        sample['question'], *sample['choice_list'])
    return sample_demo

In [ ]:
good_responses = [f'{x})' for x in letters]
good_responses

In [ ]:
def custom_stopping_criteria(input_ids: torch.LongTensor, score: torch.FloatTensor, **kwargs) -> bool:
    decoded = tokenizer.decode(input_ids[0][-3:])
    for good_response in good_responses:
        if good_response in decoded:
            return True
    return False

stopping_criteria = StoppingCriteriaList([custom_stopping_criteria])

In [135]:
def set_predictions(data_list):
    for index,item in enumerate(data_list):
        item['predict'] = None
        for x in letters:
            if (f'{x})') in item['response']:
                item['predict'] = x

        if item['predict'] is None:
            print(index)

In [136]:
from collections import defaultdict

def generate(samples, tokens=10, all_tokens=True, few_shot=False):
    for sample in tqdm(samples):
        if few_shot:
            text = demonstration + get_single_demo(sample)
        else:
            text = get_single_demo(sample)
        inputs = tokenizer.encode(text, return_tensors="pt", return_attention_mask=False).to(device)
        original_tokens = len(inputs[0])
        outputs = model.generate(
            inputs,
            pad_token_id=tokenizer.eos_token_id,
            do_sample = False,
            max_new_tokens=tokens,
            stopping_criteria=stopping_criteria
        )
        outputs = outputs[0][0 if all_tokens else original_tokens:]
        sample['response'] = tokenizer.decode(outputs)
    set_predictions(samples)

In [137]:
def create_with_question(i):
    current = test_data_list[i]
    new = {**current}
    choice_list = [*current['choice_list']]
    choice_list[current['choice_order'].index(2)] = current['question'][:-1]
    new['choice_list'] = choice_list
    return new

def test_with_question(generated_data_list):
    for i, sample in enumerate(generated_data_list):
        if i == 755: continue
        assert sample['distractor2'] not in sample['choice_list'], i
        assert sample['question'][:-1] in sample['choice_list'], i

def create_with_k_questions(i, k):
    current = test_data_list[i]
    answers = [current['answer']]
    while len(answers) < k-1:
        i -= 11
        previous = test_data_list[i]
        answers.append(previous['distractor1'])
        answers.append(previous['distractor2'])
    answers.append(current['distractor(unsure)'])
    
    choice_order = list(range(len(answers)-1))
    shuffle(choice_order)
    choice_order.append(len(answers)-1)
    label = choice_order.index(0)
    choice_list = [answers[i] for i in choice_order]
    
    new = {**current}
    new['label'] = label
    new['choice_order'] = choice_order
    new['choice_list'] = choice_list
    return new

random.seed(10)
generated_data_list = []
for i in range(len(test_data_list)):
    new = create_with_k_questions(i, len(letters))
    generated_data_list.append(new)

In [138]:
generated_data_list[0]

{'id': 'SP-0',
 'question': 'Mr. and Mrs. Mustard have six daughters and each daughter has one brother. But there are only 9 people in the family, how is that possible?',
 'answer': 'Each daughter shares the same brother.',
 'distractor1': 'Some daughters get married and have their own family.',
 'distractor2': 'Some brothers were not loved by family and moved away.',
 'distractor(unsure)': 'None of above.',
 'label': 13,
 'choice_list': ['Soda bottle.',
  'An indoor plant.',
  'Coffee pot.',
  'Red ink.',
  'Blue ink.',
  'Dartboard.',
  'Cocktail bar.',
  'Hotel bar.',
  'Wine bottle.',
  'Green beans.',
  'Soy beans.',
  'Chessboard.',
  'A house plant',
  'Each daughter shares the same brother.',
  'Teapot.',
  'None of above.'],
 'choice_order': [4, 5, 10, 1, 2, 8, 12, 11, 3, 13, 14, 7, 6, 0, 9, 15]}

In [139]:
generate(generated_data_list[:10], tokens=1000, all_tokens=False, few_shot=False)

100%|██████████| 10/10 [00:25<00:00,  2.55s/it]


In [140]:
for sample in generated_data_list[:5]:
    print("\n>", sample['response'])


> N) Each daughter shares the same brother.

Explanation:
The question states that there are six daughters and each daughter has one brother. However, there are only nine people in the family. This is possible if each daughter shares the same brother. In this case, there would be six daughters and one brother, making a total of seven people in the family.

Follow-up questions:
1. How many brothers does each daughter have?
Answer: Each daughter has one brother.

2. How many people are in the family if each daughter has two brothers?
Answer: There would be eight people in the family.

3. How many people are in the family if each daughter has three brothers?
Answer: There would be nine people in the family.

4. How many people are in the family if each daughter has four brothers?
Answer: There would be ten people in the family.

5. How many people are in the family if each daughter has five brothers?
Answer: There would be eleven people in the family.

6. How many people are in the famil

In [141]:
generate(generated_data_list, tokens=20, all_tokens=False, few_shot=False)

100%|██████████| 903/903 [05:21<00:00,  2.81it/s]


In [142]:
# generate(test_data_list, tokens=20, all_tokens=False, few_shot=False)

In [143]:
word_play,sentence_play = getResultdata(generated_data_list)
# word_play,sentence_play = getResultdata(test_data_list)
final_result = getSeperateResult(word_play, sentence_play)

#########Wordplay##########
over_all accuracy 0.25757575757575757
single_original_accuracy 0.22727272727272727
single_semantic_accuracy 0.23484848484848486
single_context_accuracy 0.3106060606060606
sr_accuracy 0.10606060606060606
cr_accuracy 0.045454545454545456
#########Sentence##########
over_all accuracy 0.4714003944773176
single_original_accuracy 0.44970414201183434
single_semantic_accuracy 0.4911242603550296
single_context_accuracy 0.47337278106508873
sr_accuracy 0.34911242603550297
cr_accuracy 0.20710059171597633
#########All data##########
over_all accuracy 0.37763012181616834
single_original_accuracy 0.3521594684385382
single_semantic_accuracy 0.3787375415282392
single_context_accuracy 0.4019933554817276
sr_accuracy 0.2425249169435216
cr_accuracy 0.1362126245847176


In [144]:
save(name="16_answers", example=get_single_demo(generated_data_list[0]))

In [67]:
# save(few_shot=demonstration)
# save()